<a href="https://colab.research.google.com/github/albim72/ML_ZAAWANSOWANY_11/blob/main/DCGAN_mnist_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from  __future__ import absolute_import
from  __future__ import division
from  __future__ import print_function

In [6]:
from tensorflow.keras.layers import Activation,Dense,Input
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import Reshape,Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

In [9]:
import numpy as np
import math
import matplotlib.pyplot
import os
import argparse

In [8]:
def build_generator(inputs,labels,image_size):
  """
  konstruowanie modelu generatora
  wejścia są łączone przed warstwą gęstą
  Stos warstw BN-Relu-Cov2DTranspose do generowania fałszywych obrazów
  funkcja aktywacji - sigmoid

  Argumenty:
  inputs  - warstwa wejściowa genezarora - wektor z
  labels - warstwa wejściowa dla wektora OH - nałożenie warunków wejścia
  image_size - docelowy rozmiar jednego bloku (kwadrat)

  Wyjście -> model generatora
  """

  image_resize = image_size//4
  kernel_size = 5
  layer_filters = [128,64,32,1]

  x = concatenate([inputs,labels],axis=1)
  x = Dense(image_resize*image_resize*layer_filters[0])(x)
  x = Reshape((image_resize,image_size,layer_filters[0]))(x)
  for filters in layer_filters:
    if filters>layer_filters[-2]:
      strides=2
    else:
      strides=1

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(filters=filters,
                        kernel_size=kernel_size,
                        strides = strides,
                        padding='same')(x)
  x = Activation('sigmoid')(x)
  generator = Model([inputs,labels],x,name='generator')
  return generator

In [10]:
def build_discriminator(inputs,labels,image_size):
  """
  konstruowanie modelu dyskryminatora
  wejścia są łączone w warstwie gęstej
  Stos warstw LeakyRelu - Conv2D do odróżniania prawdziwych i fałszywych obrazów

  Argumenty:
  inputs  - warstwa wejściowa dyskryminatora - wektor z
  labels - warstwa wejściowa dla wektora OH - nałożenie warunków wejścia
  image_size - docelowy rozmiar jednego bloku (kwadrat)

  Wyjście -> model dyskryminatora
  """
  kernel_size = 5
  layer_filters = [32,64,128,256]

  x = inputs
  y = Dense(image_size*image_size)(labels)
  y = Reshape((image_size,image_size,1))(y)
  x = concatenate([x,y])

  for filters in layer_filters:
    if filters == layer_filters[-1]:
      strides = 1
    else:
      strides = 2
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,
               padding='same')(x)

  x = Flatten()(x)
  x = Dense(1)(x)
  x = Activation('sigmoid')(x)
  discriminator = Model([inputs,labels],x,name='discriminator')
  return discriminator